<a href="https://colab.research.google.com/github/Venchislav/Data-Science/blob/main/Statube.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **GETTING COMMENTS OF VIDEO BY YOUTUBE API**

In [ ]:
!pip install apiclient

In [ ]:
from info import KEY

In [ ]:
from googleapiclient.discovery import build

api_key = KEY

coms = []
sussy_baka = 0
def video_comments(video_id):
    global sussy_baka


    # creating youtube resource object
    youtube = build('youtube', 'v3',
                    developerKey=api_key)

    # retrieve youtube video results
    video_response=youtube.commentThreads().list(
    part='snippet,replies',
    videoId=video_id
    ).execute()

    # iterate video response
    while video_response:

        # extracting required info
        # from each result object
        for item in video_response['items']:

            # Extracting comments
            comment = item['snippet']['topLevelComment']['snippet']['textDisplay']
            coms.append(comment)



            if sussy_baka < 10:
              print(comment,  end ='\n')
            sussy_baka += 1

        if sussy_baka > 50_000:
          break
        # Again repeat
        if 'nextPageToken' in video_response:
            video_response = youtube.commentThreads().list(
                    part = 'snippet,replies',
                    videoId = video_id,
                      pageToken = video_response['nextPageToken']
                ).execute()
        else:
            break

# Enter video id
video_id = "lEtP2nTsfM8"

# Call function
video_comments(video_id)

HttpError: ignored

In [ ]:
len(coms)

# **HERE ML PART BEGINS**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df = pd.read_csv('/content/sample_data/youtoxic_english_1000.csv')
df.head()

In [ ]:
df.drop(['CommentId', 'VideoId', 'IsThreat',	'IsAbusive', 'IsProvocative',	'IsObscene',	'IsHatespeech',	'IsRacist',	'IsNationalist',	'IsSexist',	'IsHomophobic',	'IsReligiousHate',	'IsRadicalism'], axis=1, inplace=True)

In [ ]:
df.head()

In [ ]:
df['IsToxic'] = df['IsToxic'].apply(lambda x: 1 if x else 0)

In [ ]:
df.head()

In [ ]:
df['Text'] = df['Text'].apply(lambda x: x.strip())

In [ ]:
df.head()

In [ ]:
train_df, test_df = train_test_split(df, test_size=300)

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
import nltk
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
nltk.download('punkt')
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import precision_score, recall_score, precision_recall_curve
from matplotlib import pyplot as plt
import numpy as np

In [ ]:
nltk.download('stopwords')

In [ ]:
snowball = SnowballStemmer(language="english")
stop_words = stopwords.words("english")

def tokenize_sentence(sentence: str, remove_sw: bool = True):
    tokens = word_tokenize(sentence, language="english")
    tokens = [i for i in tokens if i not in string.punctuation]
    if remove_sw:
        tokens = [i for i in tokens if i not in stop_words]
    tokens = [snowball.stem(i) for i in tokens]
    return tokens

In [ ]:
tokenize_sentence(train_df.iloc[0]['Text'])

In [ ]:
vectorizer = TfidfVectorizer(tokenizer=lambda x: tokenize_sentence(x, remove_sw=True))

features = vectorizer.fit_transform(train_df['Text'])

In [ ]:
model = LogisticRegression(random_state=0)
model.fit(features, train_df['Text'])

In [ ]:
model = LogisticRegression(random_state=0)
model.fit(features, train_df['IsToxic'])

In [ ]:
model.predict(features[0])

In [ ]:
train_df["Text"].iloc[0]

In [ ]:
model_pipeline = Pipeline([
    ("vectorizer", TfidfVectorizer(tokenizer=lambda x: tokenize_sentence(x, remove_sw=True))),
    ("model", LogisticRegression(random_state=0))
]
)

In [ ]:
model_pipeline.fit(train_df['Text'], train_df['IsToxic'])

In [ ]:
model_pipeline.predict(['Breaking Bad [NG+] (Alternate Ending)'])

In [ ]:
coms = [x.strip() for x in coms]

In [ ]:
toxic = 0
normal = 0
for i in coms:
  if model_pipeline.predict([i]) == [1]:
    toxic += 1
  else:
    normal += 1


In [ ]:
fig, ax = plt.subplots()
ax.pie([toxic, normal], labels=['Toxic', 'Usual'], autopct='%.1f%%')